In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine
import datetime
from tqdm.notebook import tqdm
import numpy as np

## EXTRACT
Store datasets into DataFrames

In [2]:
csv_data = "../webscraping/miami_cl_webscrape.csv"
miami_cl_df = pd.read_csv(csv_data)

In [3]:
miami_cl_df

,Unnamed: 0,datetimes,hoods,prices,bedrooms,bathrooms,sqft,housing_type,laundry,parking,cats,dogs,furnished,titles,descriptions,links
0,0,2021-05-30 14:02,(Dadeland 2/2),"$1,900",2BR,2Ba,none,apartment,w/d in unit,carport,none,none,none,●► 2/2 DADELAND: AMAZING UNIT Unique opportun...,\n\nQR Code Link to This Post\n\n\nPROP # 07K8...,https://miami.craigslist.org/mdc/apa/d/2-dadel...
1,1,2021-05-30 14:02,(Doral),"$2,557",2BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,DEPÓSITO DE $ 500,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-d...
2,2,2021-05-30 14:02,(Miami),"$2,241",2BR,2Ba,1106,apartment,w/d in unit,off-street parking,yes,yes,none,2 Bed 2 Bathroom Apartment,"\n\nQR Code Link to This Post\n\n\nOne, Two, a...",https://miami.craigslist.org/mdc/apa/d/miami-b...
3,3,2021-05-30 14:02,(Doral),"$3,399",3BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,"$500 deposit one,two,and three bedroom 24 hour...",\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-5...
4,4,2021-05-30 14:01,(doral),"$1,978",1BR,1Ba,none,apartment,w/d in unit,attached garage,none,none,none,un mes GRATIS,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-u...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2594,2594,2021-05-27 15:57,(5350 NW 84th Ave #413),"$7,500",3BR,3Ba,none,apartment,w/d in unit,detached garage,none,none,none,Beautiful turn-key furnished 3 bedroom unit wi...,\n\nQR Code Link to This Post\n\n\nBeautiful t...,https://miami.craigslist.org/mdc/apa/d/miami-b...
2595,2595,2021-05-27 15:54,(480 NE 31 St #4907),"$8,500",3BR,3Ba,none,house,w/d in unit,street parking,none,none,none,**Amazing Penthouse level 3 bedrooms and 3 bat...,\n\nQR Code Link to This Post\n\n\napartment f...,https://miami.craigslist.org/mdc/apa/d/miami-a...
2596,2596,2021-05-27 15:54,(Kendall 2/2),"$1,500",2BR,2Ba,none,apartment,w/d in unit,carport,none,none,none,●► 2/2 2 Bedroom 2 Bathroom Unit Available KE...,\n\nQR Code Link to This Post\n\n\nPROP # 06K0...,https://miami.craigslist.org/mdc/apa/d/bedroom...
2597,2597,2021-05-27 15:54,(Fontainebleu 2/2.5),"$1,700",2BR,2.5Ba,none,apartment,w/d in unit,carport,none,none,none,●► 2/2.5 FONTAINEBLEU: MAGNIFICENT UNIT Great...,\n\nQR Code Link to This Post\n\n\nPROP # 07K0...,https://miami.craigslist.org/mdc/apa/d/fontain...


## TRANSFORM
Clean DataFrame

In [4]:
miami_df = miami_cl_df.drop(columns=["Unnamed: 0"])
miami_df.head()

,datetimes,hoods,prices,bedrooms,bathrooms,sqft,housing_type,laundry,parking,cats,dogs,furnished,titles,descriptions,links
0,2021-05-30 14:02,(Dadeland 2/2),"$1,900",2BR,2Ba,none,apartment,w/d in unit,carport,none,none,none,●► 2/2 DADELAND: AMAZING UNIT Unique opportun...,\n\nQR Code Link to This Post\n\n\nPROP # 07K8...,https://miami.craigslist.org/mdc/apa/d/2-dadel...
1,2021-05-30 14:02,(Doral),"$2,557",2BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,DEPÓSITO DE $ 500,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-d...
2,2021-05-30 14:02,(Miami),"$2,241",2BR,2Ba,1106,apartment,w/d in unit,off-street parking,yes,yes,none,2 Bed 2 Bathroom Apartment,"\n\nQR Code Link to This Post\n\n\nOne, Two, a...",https://miami.craigslist.org/mdc/apa/d/miami-b...
3,2021-05-30 14:02,(Doral),"$3,399",3BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,"$500 deposit one,two,and three bedroom 24 hour...",\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-5...
4,2021-05-30 14:01,(doral),"$1,978",1BR,1Ba,none,apartment,w/d in unit,attached garage,none,none,none,un mes GRATIS,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-u...


In [5]:
miami_df = miami_df.rename(columns = {"datetimes": "datetime", "hoods": "neighborhood", "prices": "rent",\
                                                "bedrooms": "br", "bathrooms": "ba", "housing_type": "housingType",\
                                                "titles": "title", "descriptions": "description", "links": "link"})
miami_df.head()

,datetime,neighborhood,rent,br,ba,sqft,housingType,laundry,parking,cats,dogs,furnished,title,description,link
0,2021-05-30 14:02,(Dadeland 2/2),"$1,900",2BR,2Ba,none,apartment,w/d in unit,carport,none,none,none,●► 2/2 DADELAND: AMAZING UNIT Unique opportun...,\n\nQR Code Link to This Post\n\n\nPROP # 07K8...,https://miami.craigslist.org/mdc/apa/d/2-dadel...
1,2021-05-30 14:02,(Doral),"$2,557",2BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,DEPÓSITO DE $ 500,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-d...
2,2021-05-30 14:02,(Miami),"$2,241",2BR,2Ba,1106,apartment,w/d in unit,off-street parking,yes,yes,none,2 Bed 2 Bathroom Apartment,"\n\nQR Code Link to This Post\n\n\nOne, Two, a...",https://miami.craigslist.org/mdc/apa/d/miami-b...
3,2021-05-30 14:02,(Doral),"$3,399",3BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,"$500 deposit one,two,and three bedroom 24 hour...",\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-5...
4,2021-05-30 14:01,(doral),"$1,978",1BR,1Ba,none,apartment,w/d in unit,attached garage,none,none,none,un mes GRATIS,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-u...


In [6]:
# drop duplicate rows
miami_df = miami_df.drop_duplicates(subset='link')
len(miami_df.drop_duplicates(subset='link'))

2597

In [7]:
# drop duplicate rows
miami_df = miami_df.drop_duplicates(subset='title')
len(miami_df.drop_duplicates(subset='title'))

2104

In [8]:
miami_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2104 entries, 0 to 2598
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   datetime      2104 non-null   object
 1   neighborhood  2104 non-null   object
 2   rent          2104 non-null   object
 3   br            2104 non-null   object
 4   ba            2104 non-null   object
 5   sqft          2104 non-null   object
 6   housingType   2104 non-null   object
 7   laundry       2104 non-null   object
 8   parking       2104 non-null   object
 9   cats          2104 non-null   object
 10  dogs          2104 non-null   object
 11  furnished     2104 non-null   object
 12  title         2104 non-null   object
 13  description   2104 non-null   object
 14  link          2104 non-null   object
dtypes: object(15)
memory usage: 263.0+ KB


In [9]:
# convert string to datetime

miami_df['datetime'] = pd.to_datetime(miami_df['datetime'])
miami_df['datetime']

0      2021-05-30 14:02:00
1      2021-05-30 14:02:00
2      2021-05-30 14:02:00
3      2021-05-30 14:02:00
4      2021-05-30 14:01:00
               ...        
2594   2021-05-27 15:57:00
2595   2021-05-27 15:54:00
2596   2021-05-27 15:54:00
2597   2021-05-27 15:54:00
2598   2021-05-27 15:54:00
Name: datetime, Length: 2104, dtype: datetime64[ns]

In [10]:
miami_df['rent'] = miami_df['rent'].str.replace(',', '')
miami_df['rent'] = miami_df['rent'].str.replace('$', '')
miami_df['rent'] = pd.to_numeric(miami_df['rent'])
miami_df['rent']

0       1900
1       2557
2       2241
3       3399
4       1978
        ... 
2594    7500
2595    8500
2596    1500
2597    1700
2598    1850
Name: rent, Length: 2104, dtype: int64

In [11]:
# set min and max for rent and drop rows

miami_df = miami_df.loc[(miami_df['rent'] > 499.00) & (miami_df['rent'] < 4999.00)]

In [12]:
pd.set_option('display.max_rows', None)
miami_df

,datetime,neighborhood,rent,br,ba,sqft,housingType,laundry,parking,cats,dogs,furnished,title,description,link
0,2021-05-30 14:02:00,(Dadeland 2/2),1900,2BR,2Ba,none,apartment,w/d in unit,carport,none,none,none,●► 2/2 DADELAND: AMAZING UNIT Unique opportun...,\n\nQR Code Link to This Post\n\n\nPROP # 07K8...,https://miami.craigslist.org/mdc/apa/d/2-dadel...
1,2021-05-30 14:02:00,(Doral),2557,2BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,DEPÓSITO DE $ 500,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-d...
2,2021-05-30 14:02:00,(Miami),2241,2BR,2Ba,1106,apartment,w/d in unit,off-street parking,yes,yes,none,2 Bed 2 Bathroom Apartment,"\n\nQR Code Link to This Post\n\n\nOne, Two, a...",https://miami.craigslist.org/mdc/apa/d/miami-b...
3,2021-05-30 14:02:00,(Doral),3399,3BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,"$500 deposit one,two,and three bedroom 24 hour...",\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-5...
4,2021-05-30 14:01:00,(doral),1978,1BR,1Ba,none,apartment,w/d in unit,attached garage,none,none,none,un mes GRATIS,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-u...
5,2021-05-30 14:01:00,(Doral),2257,2BR,2Ba,none,apartment,w/d in unit,street parking,none,none,none,Depósito de $ 500,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-d...
6,2021-05-30 14:01:00,(Doral),1978,1BR,1Ba,none,apartment,w/d in unit,attached garage,none,none,none,"mudanza rápida en 1, 2 y 3 habitaciones",\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-m...
7,2021-05-30 14:01:00,(Doral),2557,2BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,Apartamentos nuevos extranjeros bienvenidos,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-a...
8,2021-05-30 14:01:00,(Doral),2257,2BR,2Ba,none,apartment,w/d in unit,attached garage,none,none,none,UN MES GRATIS,\n\nQR Code Link to This Post\n\n\n*** UNFURNI...,https://miami.craigslist.org/mdc/apa/d/miami-u...
9,2021-05-30 14:01:00,(North Miami Beach),1350,1BR,1Ba,600,apartment,laundry on site,valet parking,none,none,none,Newly Renovated 1 Bedroom / 1 Bath,\n\nQR Code Link to This Post\n\n\nNewly renov...,https://miami.craigslist.org/mdc/apa/d/miami-n...


In [13]:
miami_df.describe()

,rent
count,1896.000000
mean,2030.006329
std,820.117562
min,500.000000
25%,1500.000000
50%,1800.000000
75%,2300.000000
max,4950.000000


In [14]:
miami_df['br'] = miami_df['br'].str.replace('BR', '')
miami_df['br'] = pd.to_numeric(miami_df['br'])

In [15]:
miami_df['ba'] = miami_df['ba'].str.replace('sharedBa', '1')
miami_df['ba'] = miami_df['ba'].str.replace('Ba', '')
miami_df['ba'] = pd.to_numeric(miami_df['ba'])

In [16]:
miami_df['sqft'] = miami_df['sqft'].str.replace('none', '')
miami_df['sqft'] = pd.to_numeric(miami_df['sqft'])

In [17]:
miami_df['cats'] = miami_df['cats'].str.replace('none', 'no')

In [18]:
miami_df['dogs'] = miami_df['dogs'].str.replace('none', 'no')
miami_df['dogs'].value_counts()

no     1444
yes     452
Name: dogs, dtype: int64

In [19]:
miami_df['furnished'] = miami_df['furnished'].str.replace('none', 'no')
miami_df['furnished'].value_counts()

no     1805
yes      91
Name: furnished, dtype: int64

In [20]:
miami_df['title'] = miami_df['title'].str.replace('●►  ', '')
miami_df['title'] = miami_df['title'].str.replace('*', '')
miami_df['title'] = miami_df['title'].str.replace('●', '')
miami_df['title']

0       2/2 DADELAND: AMAZING UNIT Unique opportunity ...
1                                       DEPÓSITO DE $ 500
2                              2 Bed 2 Bathroom Apartment
3       $500 deposit one,two,and three bedroom 24 hour...
4                                           un mes GRATIS
5                                       Depósito de $ 500
6                 mudanza rápida en 1, 2 y 3 habitaciones
7             Apartamentos nuevos extranjeros bienvenidos
8                                           UN MES GRATIS
9                      Newly Renovated 1 Bedroom / 1 Bath
11      BRAND NEW BUILDING  TWO MONTHS FREE  ONLY $500...
12      2/2 Huge 2 bedroom with over sized l DADELAND:...
13      2/2 HAMMOCKS: THIS UNIT IS READY Rent this bea...
14      2/2 MUST SEE THIS RENTAL KENDALL: Spacious con...
15       FIRST MONTH FREE  ONLY $99 DEPOSIT  FAST APPR...
16      2/2 FONTAINEBLEU: RENT THIS LARGE 2/2 CONDO CL...
17      Separate Dining Room, Boutique Building, Fitne...
18            

In [21]:
# drop duplicate rows after cleaning up titles
miami_df = miami_df.drop_duplicates(subset='title')
len(miami_df.drop_duplicates(subset='title'))

1889

In [22]:
miami_df.loc[:, 'description'] = miami_df['description'].str.replace('\n\nQR Code Link to This Post\n\n\n', '').copy()
miami_df.loc[:, 'description'] = miami_df['description'].str.replace('\n', '').copy()
miami_df.loc[:, 'description'] = miami_df['description'].str.replace('PROP # 07K', '').copy()
miami_df.loc[:, 'description'] = miami_df['description'].str.replace('PROP # 06K', '').copy()
miami_df['description']


# netc = netb.loc[netb["DeltaAMPP"] == 0].copy()

/Users/valerie/opt/anaconda3/envs/pythondata/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


0       80DADELAND: 2/2Unique opportunity to lease thi...
1       *** UNFURNISHED ONE,TWO,AND THREE BEDROOM ****...
2       One, Two, and Three Bedroom Miami Apartments A...
3       *** UNFURNISHED ONE,TWO,AND THREE BEDROOM ****...
4       *** UNFURNISHED ONE,TWO,AND THREE BEDROOM ****...
5       *** UNFURNISHED ONE,TWO,AND THREE BEDROOM ****...
6       *** UNFURNISHED ONE,TWO,AND THREE BEDROOM ****...
7       *** UNFURNISHED ONE,TWO,AND THREE BEDROOM ****...
8       *** UNFURNISHED ONE,TWO,AND THREE BEDROOM ****...
9       Newly renovated large bedroom freshly painted ...
11      *** UNFURNISHED THREE BEDROOM ****** MINIMUM O...
12      90DADELAND: 2/2Huge 2 bedroom with over sized ...
13      28HAMMOCKS: 2/2Rent this beautiful large 2/2 c...
14      38KENDALL: 2/2Spacious condo in the beautifull...
15      *** UNFURNISHED TWO BEDROOM ****** MINIMUM ONE...
16      08FONTAINEBLEU: 2/2RENT THIS LARGE 2/2 CONDO C...
17      IN-PERSON TOURS AVAILABLE We are now welcoming...
18      DISPON

In [23]:
miami_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1889 entries, 0 to 2598
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   datetime      1889 non-null   datetime64[ns]
 1   neighborhood  1889 non-null   object        
 2   rent          1889 non-null   int64         
 3   br            1889 non-null   int64         
 4   ba            1889 non-null   float64       
 5   sqft          638 non-null    float64       
 6   housingType   1889 non-null   object        
 7   laundry       1889 non-null   object        
 8   parking       1889 non-null   object        
 9   cats          1889 non-null   object        
 10  dogs          1889 non-null   object        
 11  furnished     1889 non-null   object        
 12  title         1889 non-null   object        
 13  description   1889 non-null   object        
 14  link          1889 non-null   object        
dtypes: datetime64[ns](1), float64(2), int6

In [24]:
miami_df.loc[:, 'neighborhood'] = miami_df['neighborhood'].str.title().copy()
miami_df['neighborhood'].value_counts()

 (Kendall 2/2)                                                     236
 (Miami Beach)                                                     167
 (Miami)                                                           139
 (Fontainebleu 2/2)                                                 98
 (Coral Gables)                                                     89
 (Dadeland 2/2)                                                     75
 (Hammocks 2/2)                                                     69
 (Doral)                                                            57
 (Brickell)                                                         56
 (North Miami)                                                      46
 (Kendall)                                                          46
 (North Miami Beach)                                                28
 (South Dadeland 2/2)                                               26
 (Miami Beach , South Beach)                                        25
 (Font

In [25]:
# data cleaning neighborhood column

miami_df.loc[:, 'neighborhood'] = miami_df['neighborhood'].str.replace('(', '')
miami_df.loc[:, 'neighborhood'] = miami_df['neighborhood'].str.replace(')', '')
miami_df.loc[:, 'neighborhood'] = miami_df['neighborhood'].str.lstrip()
miami_df.loc[:, 'neighborhood'] = miami_df['neighborhood'].str.replace(', FL', '')
miami_df.loc[:, 'neighborhood'] = miami_df['neighborhood'].str.replace(', Fl', '')

miami_df.loc[miami_df['neighborhood'].str.contains('Brickell'), 'neighborhood'] = 'Brickell'
miami_df.loc[miami_df['neighborhood'].str.contains('Brickll'), 'neighborhood'] = 'Brickell'
miami_df.loc[miami_df['neighborhood'].str.contains('Bricekll'), 'neighborhood'] = 'Brickell'

miami_df.loc[miami_df['neighborhood'].str.contains('Kkendall'), 'neighborhood'] = 'Kendall'
miami_df.loc[miami_df['neighborhood'].str.contains('Kendall'), 'neighborhood'] = 'Kendall'
miami_df.loc[miami_df['neighborhood'].str.contains('Hammocks'), 'neighborhood'] = 'Kendall'
miami_df.loc[miami_df['neighborhood'].str.contains('Dadeland'), 'neighborhood'] = 'Dadeland'

miami_df.loc[miami_df['neighborhood'].str.contains('South Beach'), 'neighborhood'] = 'South Beach'
miami_df.loc[miami_df['neighborhood'].str.contains('Miami Beach'), 'neighborhood'] = 'Miami Beach'


miami_df.loc[miami_df['neighborhood'].str.contains('Edgewater'), 'neighborhood'] = 'Midtown / Edgewater'
miami_df.loc[miami_df['neighborhood'].str.contains('Midtown'), 'neighborhood'] = 'Midtown / Edgewater'
miami_df.loc[miami_df['neighborhood'].str.contains('Midtown'), 'neighborhood'] = 'Midtown / Edgewater'
miami_df.loc[miami_df['neighborhood'].str.contains('Midtown'), 'neighborhood'] = 'Midtown / Edgewater'
miami_df.loc[miami_df['neighborhood'].str.contains('Downtown'), 'neighborhood'] = 'Downtown Miami'

miami_df.loc[miami_df['neighborhood'].str.contains('Deerwod'), 'neighborhood'] = 'Deerwood'

miami_df.loc[miami_df['neighborhood'].str.contains('Hialeah'), 'neighborhood'] = 'Hialeah'

miami_df.loc[miami_df['neighborhood'].str.contains('Aventura'), 'neighborhood'] = 'Aventura'
miami_df.loc[miami_df['neighborhood'].str.contains('Sunny Isles'), 'neighborhood'] = 'Sunny Isles Beach'

miami_df.loc[miami_df['neighborhood'].str.contains('Fontainebleu'), 'neighborhood'] = 'Fontainebleau'
miami_df.loc[miami_df['neighborhood'].str.contains('Fontainbleau'), 'neighborhood'] = 'Fontainebleau'

miami_df.loc[miami_df['neighborhood'].str.contains('Wynwood'), 'neighborhood'] = 'Wynwood'
miami_df.loc[miami_df['neighborhood'].str.contains('Wymnwood'), 'neighborhood'] = 'Wynwood'

miami_df.loc[miami_df['neighborhood'].str.contains('Coral Gables'), 'neighborhood'] = 'Coral Gables'

miami_df.loc[miami_df['neighborhood'].str.contains('Cutler Bay'), 'neighborhood'] = 'Cutler Bay'

miami_df.loc[miami_df['neighborhood'].str.contains('Doral'), 'neighborhood'] = 'Doral'

miami_df.loc[miami_df['neighborhood'].str.contains('Boca Raton'), 'neighborhood'] = 'Boca Raton'

miami_df.loc[miami_df['neighborhood'].str.contains('Homestead'), 'neighborhood'] = 'Homestead'

miami_df.loc[miami_df['neighborhood'].str.contains('Westchester'), 'neighborhood'] = 'Westchester'

miami_df.loc[miami_df['neighborhood'].str.contains('Little Havana'), 'neighborhood'] = 'Little Havana'
miami_df.loc[miami_df['neighborhood'].str.contains('Little Havanna'), 'neighborhood'] = 'Little Havana'
miami_df.loc[miami_df['neighborhood'].str.contains('North Miami'), 'neighborhood'] = 'North Miami'
miami_df.loc[miami_df['neighborhood'].str.contains('Miami Garden'), 'neighborhood'] = 'Miami Gardens'
miami_df.loc[miami_df['neighborhood'].str.contains('Miami Shores'), 'neighborhood'] = 'Miami Shores'
miami_df.loc[miami_df['neighborhood'].str.contains('Opalocka'), 'neighborhood'] = 'Opa Locka'
miami_df.loc[miami_df['neighborhood'].str.contains('Palmetto Bay'), 'neighborhood'] = 'Palmetto Bay'
miami_df.loc[miami_df['neighborhood'].str.contains('Pinecrest'), 'neighborhood'] = 'Pinecrest'
miami_df.loc[miami_df['neighborhood'].str.contains('Sweetwater'), 'neighborhood'] = 'Sweetwater'


# miami_df

# miami_df['neighborhood'] = miami_df['neighborhood'].str.replace(' 2/2.5', '')
# miami_df['neighborhood'] = miami_df['neighborhood'].str.replace(' 2/2', '')

miami_df.loc[miami_df['neighborhood'].str.contains('Collins'), 'neighborhood'] = 'Miami Beach'
miami_df.loc[miami_df['neighborhood'].str.contains('Alton'), 'neighborhood'] = 'Miami Beach'
miami_df.loc[miami_df['neighborhood'].str.contains('Venetian'), 'neighborhood'] = 'Miami Beach'
miami_df.loc[miami_df['neighborhood'].str.contains('Meridian'), 'neighborhood'] = 'Miami Beach'
miami_df.loc[miami_df['neighborhood'].str.contains('Lincoln'), 'neighborhood'] = 'Miami Beach'
miami_df.loc[miami_df['neighborhood'].str.contains('Drexel'), 'neighborhood'] = 'Miami Beach'
miami_df.loc[miami_df['neighborhood'].str.contains('Ocean'), 'neighborhood'] = 'Miami Beach'
miami_df.loc[miami_df['neighborhood'].str.contains('Surfside'), 'neighborhood'] = 'Miami Beach'
miami_df.loc[miami_df['neighborhood'].str.contains('Bal Harbour'), 'neighborhood'] = 'Miami Beach'
miami_df.loc[miami_df['neighborhood'].str.contains('Bay Harbor Islands'), 'neighborhood'] = 'Miami Beach'
miami_df.loc[miami_df['neighborhood'].str.contains('Biscayne'), 'neighborhood'] = 'Midtown / Edgewater'
miami_df.loc[miami_df['neighborhood'].str.contains('Bayshore'), 'neighborhood'] = 'Midtown / Edgewater'
miami_df.loc[miami_df['neighborhood'].str.contains('Edgewatwer'), 'neighborhood'] = 'Midtown / Edgewater'
miami_df.loc[miami_df['neighborhood'].str.contains('Ives Dairy Rd'), 'neighborhood'] = 'Aventura'
miami_df.loc[miami_df['neighborhood'].str.contains('S Miami Ave'), 'neighborhood'] = 'Brickell'

miami_df.loc[miami_df['neighborhood'].str.contains('Tigertail'), 'neighborhood'] = 'Coconut Grove'

miami_df.loc[miami_df['neighborhood'].str.contains('Isles Beach'), 'neighborhood'] = 'Sunny Isles Beach'

miami_df.loc[miami_df['neighborhood'].str.contains('Mimo'), 'neighborhood'] = 'Mimo / Upper East Side'
miami_df.loc[miami_df['neighborhood'].str.contains('Upper East Side'), 'neighborhood'] = 'Mimo / Upper East Side'


miami_df.loc[miami_df['neighborhood'].str.contains('11325 Nw 7Th St Miami'), 'neighborhood'] = 'Fontainebleau'
miami_df.loc[miami_df['neighborhood'].str.contains('19491 Sw 210Th St'), 'neighborhood'] = 'Redland'
miami_df.loc[miami_df['neighborhood'].str.contains('8217 Sw 72 Avenue #1306E'), 'neighborhood'] = 'Dadeland'
miami_df.loc[miami_df['neighborhood'].str.contains('480 Ne 31 St #4907'), 'neighborhood'] = 'Midtown / Edgewater'
miami_df.loc[miami_df['neighborhood'].str.contains('Miami ,'), 'neighborhood'] = 'Miami'
miami_df.loc[miami_df['neighborhood'].str.contains('5350 Nw 84Th Ave'), 'neighborhood'] = 'Doral'
miami_df.loc[miami_df['neighborhood'].str.contains('7280 S Waterway Dr'), 'neighborhood'] = 'Westchester'
miami_df.loc[miami_df['neighborhood'].str.contains('9201 Sw 56Th Ter #9201'), 'neighborhood'] = 'Olympia Heights'
miami_df.loc[miami_df['neighborhood'].str.contains('9025 Garland Ave'), 'neighborhood'] = 'Surfside'
miami_df.loc[miami_df['neighborhood'].str.contains('7396 Sw 128Th St #-'), 'neighborhood'] = 'Pinecrest'
miami_df.loc[miami_df['neighborhood'].str.contains('1000 E Island Blvd #304'), 'neighborhood'] = 'Aventura'
miami_df.loc[miami_df['neighborhood'].str.contains('3500 Mystic Pointe Dr #1008'), 'neighborhood'] = 'Aventura'
miami_df.loc[miami_df['neighborhood'].str.contains('21055 Ne 37Th Ave #1602'), 'neighborhood'] = 'Aventura'
miami_df.loc[miami_df['neighborhood'].str.contains('416 Miller Rd'), 'neighborhood'] = 'Coral Gables'
miami_df.loc[miami_df['neighborhood'].str.contains('301 88Th St #0'), 'neighborhood'] = 'Surfside'
miami_df.loc[miami_df['neighborhood'].str.contains('8200 Byron Ave #A'), 'neighborhood'] = 'Miami Beach'
miami_df.loc[miami_df['neighborhood'].str.contains('Miami, Waterfront, For Rent'), 'neighborhood'] = 'Little River'
miami_df.loc[miami_df['neighborhood'].str.contains('Allapattah'), 'neighborhood'] = 'Allapattah'
miami_df.loc[miami_df['neighborhood'].str.contains('4100 Island Blvd #1402'), 'neighborhood'] = 'Aventura'
miami_df.loc[miami_df['neighborhood'].str.contains('531 Ne 68 St'), 'neighborhood'] = 'Mimo / Upper East Side'
miami_df.loc[miami_df['neighborhood'].str.contains('1796 Normandy Dr'), 'neighborhood'] = 'Miami Beach'
miami_df.loc[miami_df['neighborhood'].str.contains('6960 Nw 104Th Ct #6960'), 'neighborhood'] = 'Medley'
miami_df.loc[miami_df['neighborhood'].str.contains('3723 Ne 16Th St'), 'neighborhood'] = 'Homestead'
miami_df.loc[miami_df['neighborhood'].str.contains('6250 Sw 27Th St'), 'neighborhood'] = 'Kendall'
miami_df.loc[miami_df['neighborhood'].str.contains('6880 Sw 44Th Street'), 'neighborhood'] = 'South Miami'
miami_df.loc[miami_df['neighborhood'].str.contains('Paraiso'), 'neighborhood'] = 'Midtown / Edgewater'
miami_df.loc[miami_df['neighborhood'].str.contains('33156'), 'neighborhood'] = 'Dadeland'
miami_df.loc[miami_df['neighborhood'].str.contains('Cabana Club Apartments'), 'neighborhood'] = 'Cutler Bay'
miami_df.loc[miami_df['neighborhood'].str.contains('Coral Way'), 'neighborhood'] = 'Coral Gables'
miami_df.loc[miami_df['neighborhood'].str.contains('Andover Lakes  Estates'), 'neighborhood'] = 'Miami Gardens'
miami_df.loc[miami_df['neighborhood'].str.contains('727 Ne 73Rd St'), 'neighborhood'] = 'Mimo / Upper East Side'
miami_df.loc[miami_df['neighborhood'].str.contains('5734 Nw 12 Av'), 'neighborhood'] = 'Liberty City'
miami_df.loc[:, 'neighborhood'] = miami_df['neighborhood'].str.replace(' Fl', '')
miami_df.loc[miami_df['neighborhood'].str.contains('2901 Virginia Street'), 'neighborhood'] = 'Coconut Grove'
miami_df.loc[miami_df['neighborhood'].str.contains('Quantum On The Bay'), 'neighborhood'] = 'Midtown / Edgewater'
miami_df.loc[miami_df['neighborhood'].str.contains('3271 Sw 18Th Ter'), 'neighborhood'] = 'Coral Gables'
miami_df.loc[miami_df['neighborhood'].str.contains('Princeton'), 'neighborhood'] = 'Princeton'
miami_df.loc[miami_df['neighborhood'].str.contains('Upper Eastside'), 'neighborhood'] = 'Mimo / Upper East Side'
miami_df.loc[miami_df['neighborhood'].str.contains('Little Haiti'), 'neighborhood'] = 'Little Haiti'
miami_df.loc[miami_df['neighborhood'].str.contains('270 Nw 58 Ct'), 'neighborhood'] = 'Flagler'
miami_df.loc[miami_df['neighborhood'].str.contains('3310 Sw 72Nd Ave'), 'neighborhood'] = 'Coral Terrace'
miami_df.loc[miami_df['neighborhood'].str.contains('Kendal'), 'neighborhood'] = 'Kendall'
miami_df.loc[miami_df['neighborhood'].str.contains('Icon Bay'), 'neighborhood'] = 'Midtown / Edgewater'
miami_df.loc[miami_df['neighborhood'].str.contains('11474 Sw 109Th Rd'), 'neighborhood'] = 'Kendall'
miami_df.loc[miami_df['neighborhood'].str.contains('Dinner Key Marina'), 'neighborhood'] = 'Coconut Grove'
miami_df.loc[miami_df['neighborhood'].str.contains('Near Fiu'), 'neighborhood'] = 'Westchester'
miami_df.loc[miami_df['neighborhood'].str.contains('Ne.68 Th.'), 'neighborhood'] = 'Little Haiti'
miami_df.loc[miami_df['neighborhood'].str.contains('Unincorporated Dade County'), 'neighborhood'] = 'Miami'
miami_df.loc[miami_df['neighborhood'].str.contains('Ventura'), 'neighborhood'] = 'Aventura'
miami_df.loc[miami_df['neighborhood'].str.contains('Southorida'), 'neighborhood'] = 'Miami'
miami_df.loc[miami_df['neighborhood'].str.contains('33175'), 'neighborhood'] = 'Kendall'
miami_df.loc[miami_df['neighborhood'].str.contains('Flamingo'), 'neighborhood'] = 'South Beach'
miami_df.loc[miami_df['neighborhood'].str.contains('4930 Sw 87Th Ct'), 'neighborhood'] = 'Olympia Heights'
miami_df.loc[miami_df['neighborhood'].str.contains('Aeropuerto'), 'neighborhood'] = 'Flagler'
miami_df.loc[miami_df['neighborhood'].str.contains('Roads Area'), 'neighborhood'] = 'Shenandoah'
miami_df.loc[miami_df['neighborhood'].str.contains('Virginia Gardens'), 'neighborhood'] = 'Virginia Gardens'
miami_df.loc[miami_df['neighborhood'].str.contains('1449 Sw 18Th Ave'), 'neighborhood'] = 'Shenandoah'
miami_df.loc[miami_df['neighborhood'].str.contains('502 Ne 77 St'), 'neighborhood'] = 'Mimo / Upper East Side'

# miami_df['neighborhood'] = miami_df['neighborhood'].str.replace('Little Havana / Dowtowwn', 'Little Havana')
miami_df['neighborhood'].value_counts()

Kendall                                                      430
Miami Beach                                                  226
Miami                                                        157
Dadeland                                                     134
Fontainebleau                                                133
Coral Gables                                                 125
Brickell                                                     100
Doral                                                         61
South Beach                                                   52
Aventura                                                      52
North Miami                                                   50
Midtown / Edgewater                                           41
Downtown Miami                                                39
Sunny Isles Beach                                             29
Hialeah                                                       24
South Miami              

In [26]:
# drop rows where neighborhood is outside of miami area

non_miami = ["Pembroke Pines", "Hollywood", "Fort Lauderdale", "Hallandale Beach", "Palm Beach Gardens",\
            "North Lauderdale", "Kissimmee", "West Palm Beach", "Honolulu", "Hallandale", "Plantation", "Saratoga Springs",\
            "1606 Wiley St", "1509 Fletcher St", "3115 Calle Largo Drive", "1431 Fletcher St", "1911 Waters Edge", "6401 Grant St",\
            "2635 Liberty St", "924 Tanglewood Cir", "3657 Sw 26 St", "5112 Sw 87Th Ter", "8599 Lakeside Dr", "Zimmerman",\
            "16384 Sw 7Th Street", "1000 Nw 7Th St", "Incredibly"]

for i in tqdm(non_miami): 
    filt = miami_df[miami_df['neighborhood'].str.contains(i)]
    miami_df.drop(index = filt.index, inplace = True)
#     miami_df.loc[miami_df['neighborhood']
#     miami_df.loc[:, 'neighborhood']

  0%|          | 0/27 [00:00<?, ?it/s]

/Users/valerie/opt/anaconda3/envs/pythondata/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [27]:
miami_df.loc[:, 'neighborhood']

0                     Dadeland
1                        Doral
2                        Miami
3                        Doral
4                        Doral
5                        Doral
6                        Doral
7                        Doral
8                        Doral
9                  Miami Beach
11                       Miami
12                    Dadeland
13                     Kendall
14                     Kendall
15                 North Miami
16               Fontainebleau
17                Coral Gables
18                     Flagler
19              Downtown Miami
20      Mimo / Upper East Side
22                     Kendall
23                       Miami
24                     Kendall
25                     Kendall
26               Fontainebleau
27                     Kendall
28                     Hialeah
30                     Kendall
32                     Kendall
33                 Brownsville
34              Downtown Miami
35               Fontainebleau
36      

In [28]:
filt

,datetime,neighborhood,rent,br,ba,sqft,housingType,laundry,parking,cats,dogs,furnished,title,description,link
1664,2021-05-28 13:46:00,Incredibly,1000,2,3.0,2106.0,house,w/d in unit,attached garage,yes,yes,no,BEAUTIFUL SWIMMING POOL AND LANDSCAPING WITH I...,This property is located in beautiful Surfside...,https://miami.craigslist.org/mdc/apa/d/fort-la...


In [29]:
miami_df['neighborhood'].value_counts()

Kendall                   430
Miami Beach               226
Miami                     157
Dadeland                  134
Fontainebleau             133
Coral Gables              125
Brickell                  100
Doral                      61
Aventura                   52
South Beach                52
North Miami                50
Midtown / Edgewater        41
Downtown Miami             39
Sunny Isles Beach          29
Hialeah                    24
South Miami                22
Cutler Bay                 15
Deerwood                   14
Coconut Grove              13
North Bay Village          12
Homestead                  11
Little Havana              10
Miami Gardens              10
Mimo / Upper East Side      9
Westchester                 8
Richmond West               7
Olympia Heights             5
Sweetwater                  4
El Portal                   4
Miami Lakes                 4
Palmetto Bay                4
Wynwood                     3
Florida City                3
Little Hai

In [30]:
miami_df.index += 1; miami_df.index.name='id'
miami_df.head()

,datetime,neighborhood,rent,br,ba,sqft,housingType,laundry,parking,cats,dogs,furnished,title,description,link
id,,,,,,,,,,,,,,,
1,2021-05-30 14:02:00,Dadeland,1900,2,2.0,NaN,apartment,w/d in unit,carport,no,no,no,2/2 DADELAND: AMAZING UNIT Unique opportunity ...,80DADELAND: 2/2Unique opportunity to lease thi...,https://miami.craigslist.org/mdc/apa/d/2-dadel...
2,2021-05-30 14:02:00,Doral,2557,2,2.0,NaN,apartment,w/d in unit,attached garage,no,no,no,DEPÓSITO DE $ 500,"*** UNFURNISHED ONE,TWO,AND THREE BEDROOM ****...",https://miami.craigslist.org/mdc/apa/d/miami-d...
3,2021-05-30 14:02:00,Miami,2241,2,2.0,1106.0,apartment,w/d in unit,off-street parking,yes,yes,no,2 Bed 2 Bathroom Apartment,"One, Two, and Three Bedroom Miami Apartments A...",https://miami.craigslist.org/mdc/apa/d/miami-b...
4,2021-05-30 14:02:00,Doral,3399,3,2.0,NaN,apartment,w/d in unit,attached garage,no,no,no,"$500 deposit one,two,and three bedroom 24 hour...","*** UNFURNISHED ONE,TWO,AND THREE BEDROOM ****...",https://miami.craigslist.org/mdc/apa/d/miami-5...
5,2021-05-30 14:01:00,Doral,1978,1,1.0,NaN,apartment,w/d in unit,attached garage,no,no,no,un mes GRATIS,"*** UNFURNISHED ONE,TWO,AND THREE BEDROOM ****...",https://miami.craigslist.org/mdc/apa/d/miami-u...


In [31]:
miami_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1836 entries, 1 to 2599
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   datetime      1836 non-null   datetime64[ns]
 1   neighborhood  1836 non-null   object        
 2   rent          1836 non-null   int64         
 3   br            1836 non-null   int64         
 4   ba            1836 non-null   float64       
 5   sqft          616 non-null    float64       
 6   housingType   1836 non-null   object        
 7   laundry       1836 non-null   object        
 8   parking       1836 non-null   object        
 9   cats          1836 non-null   object        
 10  dogs          1836 non-null   object        
 11  furnished     1836 non-null   object        
 12  title         1836 non-null   object        
 13  description   1836 non-null   object        
 14  link          1836 non-null   object        
dtypes: datetime64[ns](1), float64(2), int6

In [32]:
miami_df.to_csv('miami_df_final.csv', header=True, index=True)

## LOAD
Load DataFrame

In [33]:
# rds_connection_string = "postgres:postgres@localhost:5432/miami_rentals"
# engine = create_engine(f'postgresql://{rds_connection_string}')

In [34]:
# engine.table_names()

In [35]:
# miami_df.to_sql(name='miami_rentals', con=engine, if_exists='append', index=True)

In [36]:
# pd.read_sql_query('SELECT * FROM miami_rentals', con=engine).head()